<a href="https://colab.research.google.com/github/Bread806/goldbach_backup_from_colab/blob/main/goldbach_0205_MutiLN_mod25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# mod 25 實驗結果測試
<a href="https://drive.google.com/file/d/1RzHAFqRE0sV5uXkwbzjXXo0uOHL8SFRo/view?usp=sharing">請見paper</a><br>
要對 G4 做正規化

In [ ]:
from google.colab import drive
drive.mount("/content/Drive")

Mounted at /content/Drive


In [ ]:
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding, Dense, Flatten, LayerNormalization, Input,Activation, concatenate, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from random import choice, sample
import math


def is_prime(num):
    """檢查一個數字是否為質數"""
    if num <= 1:
        return False
    sqr_num = int(num ** 0.5)
    # for i in range(2, sqr_num+ 1):
    for i in range(2, sqr_num+ 1):
        if num % i == 0:
            return False
    return True


def prime_table(x):
    """建立小於x的質數表"""
    primes = [num for num in range(2, x) if is_prime(num)]
    return primes


def convert_base_into_list(number, base, width=10):
    result = []  # init list

    for i in range(width):
        result.append(number % base)
        number = number // base
    result = result[::-1]
    return result


def prime_to_index(primeSize, primes, number):
    for index in range(primeSize):
        if number == primes[index]:
            return index
    return -1

def add_prime(primeTable, limit, addSize=5):
    halfList = [num for num in primeTable if num <= int(limit//2)]
    if len(halfList) < 5:
      halfList += [0] * (5 - len(halfList))
    return sample(halfList, addSize)

def add_mod(primeTable25, num):
  return [num%index for index in primeTable25]

def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a


def compute_coprimes(n): ## 獲得質因數 zero padding 到40
    coprimes = []
    lenth = 50
    for num in range(1,n):
        if gcd(num , n) == 1:
            coprimes.append(num)
        if len(coprimes) >= lenth:
            return coprimes
    coprimes = coprimes + [0] * (lenth - len(coprimes))
    return coprimes


In [ ]:
# shuffle

print("---loading data---")
# load data
trainData = pd.read_csv('/content/Drive/MyDrive/實驗/goldbach/csv/traindata_DividebyG4.csv')
trainDataNumbers = trainData['Number'].values
trainDataLabel = trainData['Factors'].values
print("---loading data done.---")

# shuffle training data
shuffleIndices = np.random.permutation(len(trainDataNumbers))
shuffledNumbers = trainDataNumbers[shuffleIndices]
shuffledLabel = trainDataLabel[shuffleIndices]

---loading data---
---loading data done.---


In [ ]:
print (len(trainDataNumbers))

1048575


In [ ]:
# loadding prime table
primeTable = pd.read_csv("/content/Drive/MyDrive/實驗/goldbach/csv/prime_table_5000000.csv")
primeTableNumber = primeTable["primes"].values
print ("---loading primes table---")

---loading primes table---


In [ ]:
# generate 25 prime table
primeTable25 = primeTableNumber[:25]
print (primeTable25)

[ 2  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89
 97]


In [ ]:
# collect data
# prepare training set
mergedTrainNumber = []
mergedTrainLabel = []

for num in shuffledNumbers:

    #add 25 prefix prime
    feature = []
    feature = add_mod(primeTable25, num)
    mergedTrainNumber.append(feature)

for num in shuffledLabel:
    mergedTrainLabel.append(num)

    print("merge number ", num, " done.")

#print (mergedTrainNumber)

In [ ]:
mergedArray = np.array(mergedTrainNumber)
mergedLabel = np.array(mergedTrainLabel)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(mergedArray, mergedLabel, test_size=0.2, random_state=42)
print ("X Train [0] value : \n",X_train[0])
print ("X Train [0] label : ",y_train[0])
print ("=======")
print ("X shpae : " , X_train[0].shape)

X Train [0] value : 
 [ 0  2  2  0  2  3 16 16  6  0  7  0 29 33 38 43 19 38 56  0 32 62 12 75
 47]
X Train [0] label :  0.90454345
X shpae :  (25,)


In [ ]:
# start training
# 清空所有
K.clear_session()

################################################################

def LN_Layer(x):
  # two group
  group1 = x
  group2 = Activation('relu')(x)
  group2 = Lambda(lambda x: K.log(x + 1))(group2)  # 使用 natural logarithm

  # G1 -> next
  group1 = Dense(units=120, activation='relu')(group1)

  # G2 -> exp -> next
  group2 = Dense(units=80, activation='linear')(group2)
  group2 = Activation('exponential')(group2)

  # concate
  merged = concatenate([group1, group2], axis=-1)

  return merged


# require gpu
device = "/device:GPU:0"
with tf.device(device):
    print(f"--- gpu {device} is loaded. ---")
    inputLayer = Input(shape=X_train[0].shape)

    # 先加兩層
    x = Dense(units=25, activation='relu')(inputLayer)
    x = Dense(units=200, activation='relu')(x)

    #LN
    LNOut = LN_Layer(x)

    # 後加兩層
    x = Dense(units=200, activation='relu')(LNOut)
    x = Dense(units=200, activation='relu')(x)

    #Output
    outputLayer = Dense(units=1, activation='linear')(x)
    model = Model(inputs=inputLayer , outputs=outputLayer)


    print(model.summary())

    custom_adam_optimizer = Adam(learning_rate=0.001)
    model.compile(loss='MSE', optimizer=custom_adam_optimizer,
                  metrics=['accuracy'])    # train version 2

    # train model
    print("---starting training---")
    trainHistory = model.fit(X_train, y_train, epochs=200,batch_size=2048, validation_data=(X_val, y_val))
    print("--------------------------------------------------------")

    # save model
    #model.save('./csv/goldbach_model_1112_400w.h5')
print("model saved.")


--- gpu /device:GPU:0 is loaded. ---
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 25)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 26)                   676       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 200)                  5400      ['dense[0][0]']               
                                                                                                  
 activation (Activation)     (None, 200)                  0         ['dense_1[0][0]']             
                                                         

## predict

In [ ]:
from google.colab import drive
drive.mount("/content/Drive")

Mounted at /content/Drive


In [ ]:
# loadding prime table
primeTable = pd.read_csv("/content/Drive/MyDrive/實驗/goldbach/csv/prime_table_5000000.csv")
primeTableNumber = primeTable["primes"].values
print ("---loading primes table---")

# generate 25 prime table
primeTable25 = primeTableNumber[:25]
print (primeTable25)

---loading primes table---
[ 2  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89
 97]


In [ ]:
## load data
#predict

from tensorflow.keras.models import load_model
import csv
import pandas as pd
import numpy as np

# 指定 HDF5 模型文件的路徑
modelPath = '/content/Drive/MyDrive/實驗/model_save/goldbach_model_0205_400w.h5'

# 使用 load_model
model = load_model(modelPath)

# load test_set and prime data
testDataPath = '/content/Drive/MyDrive/實驗/goldbach/csv/test_set_G4_0.csv'
testData = pd.read_csv(testDataPath)
testDataNumbers = testData['Number'].values
testDataLabel = testData['Factors'].values
testDataG4 = testData['G4'].values
testDataPartition = testData['Partition'].values

print("---loading test data done.---")


---loading test data done.---


In [ ]:
# collect data
# prepare training set
mergedTestNumber = []
mergedTestLabel = []

for num in testDataNumbers:

    ### mod25
    feature = []
    feature = add_mod(primeTable25, num)
    mergedTestNumber.append(feature)

for num in testDataPartition:
    mergedTestLabel.append(num)

In [ ]:
mergedArray = np.array(mergedTestNumber)
mergedLabel = np.array(mergedTestLabel)

In [ ]:
print (mergedTestNumber[0])
print (mergedLabel[0])

[0, 2, 3, 1, 5, 8, 8, 5, 7, 6, 9, 3, 27, 39, 19, 26, 51, 46, 55, 38, 21, 36, 20, 4, 65]
8617


In [ ]:
evaluation = model.evaluate(mergedArray, mergedLabel)
#XNewData = testDataNumbers, testDataLabel
print ("Loss:", evaluation[0])
print ("Accuracy:", evaluation[1])
print ("##########################")
predictions = model.predict(mergedArray)



2/2 [==============================] - 0s 13ms/step - loss: 138463360.0000 - accuracy: 0.0000e+00
Loss: 138463360.0
Accuracy: 0.0
##########################
2/2 [==============================] - 0s 5ms/step


In [ ]:
print (f"測試五十筆資料")
roundPredictions = np.round(predictions)
for index, number in enumerate(testDataNumbers):
  print (f"第{index}筆資料為數字:\t{number}\t|正確答案為:{testDataPartition[index]}\t模型預估為{predictions[index]}, 四捨五入到{roundPredictions[index]}")
print (testDataNumbers[:5])
print (testDataLabel[:5])
print (predictions[:5])


# +N 才要執行正規

In [ ]:

print (f"將50筆測資正規化回G4")
normalizedTest = []
normalizedPred = []
for index in range(len(testDataLabel)):
  normalizedTest.append(testDataLabel[index]*testDataG4[index]) # normalize ground truth
  normalizedPred.append(predictions[index]*testDataG4[index])  # normalize predition


將50筆測資正規化回G4


In [ ]:
print (f"將50筆測資正規化回G4")
for index, number in enumerate(testDataNumbers):
  print (f"第{index}筆資料為數字:\t{number}\t|正確答案 = {testDataPartition[index]}\t模型預估 = {int(normalizedPred[index])}")

將50筆測資正規化回G4
第0筆資料為數字:	2413328	|正確答案 = 8617	模型預估 = 8730
第1筆資料為數字:	2063850	|正確答案 = 19867	模型預估 = 20022
第2筆資料為數字:	2127556	|正確答案 = 8004	模型預估 = 7827
第3筆資料為數字:	2738944	|正確答案 = 10361	模型預估 = 10717
第4筆資料為數字:	3396126	|正確答案 = 23485	模型預估 = 23149
第5筆資料為數字:	3285178	|正確答案 = 12582	模型預估 = 12442
第6筆資料為數字:	1222818	|正確答案 = 10215	模型預估 = 9753
第7筆資料為數字:	2319246	|正確答案 = 17107	模型預估 = 16721
第8筆資料為數字:	2093692	|正確答案 = 7593	模型預估 = 7747
第9筆資料為數字:	2549384	|正確答案 = 9229	模型預估 = 9116
第10筆資料為數字:	1991774	|正確答案 = 7271	模型預估 = 7307
第11筆資料為數字:	2938648	|正確答案 = 10311	模型預估 = 10326
第12筆資料為數字:	1310512	|正確答案 = 6145	模型預估 = 6229
第13筆資料為數字:	975824	|正確答案 = 4055	模型預估 = 4063
第14筆資料為數字:	1899278	|正確答案 = 7413	模型預估 = 7433
第15筆資料為數字:	2829450	|正確答案 = 28467	模型預估 = 28692
第16筆資料為數字:	942238	|正確答案 = 4255	模型預估 = 4308
第17筆資料為數字:	1489478	|正確答案 = 5706	模型預估 = 5724
第18筆資料為數字:	3728288	|正確答案 = 12475	模型預估 = 12639
第19筆資料為數字:	1401200	|正確答案 = 7447	模型預估 = 7268
第20筆資料為數字:	2226458	|正確答案 = 9190	模型預估 = 9413
第21筆資料為數字:	2073622	|正確答案 = 8081	模型預估 = 8198
第22筆資料為數字:	129

<ipython-input-28-1db06758a734>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print (f"第{index}筆資料為數字:\t{number}\t|正確答案 = {testDataPartition[index]}\t模型預估 = {int(normalizedPred[index])}")


In [ ]:
print ("\t正確答案  "  , "\t/" ,"  \t模型預測")
for index, number in enumerate(testDataNumbers):
  print (f"{index}\t{testDataLabel[index]}  \t\t/  \t{roundPredictions[index]} = {testDataLabel[index]/roundPredictions[index]} ")
print ("###########################################################################")

print ("\t正確答案  "  , "\t-" ,"  \t模型預測")
for index, number in enumerate(testDataNumbers):
  print (f"{index}\t{testDataLabel[index]}  \t\t-  \t{roundPredictions[index]} = {testDataLabel[index]-roundPredictions[index]} ")

In [ ]:
#正確答案 = {testDataPartition[index]}\t模型預估 = {int(normalizedPred[index])}")
D = []
for index in range (len(testDataPartition)):
  D.append(testDataPartition[index] - int(normalizedPred[index]))


print ("\t正確答案  "  , "\t-" ,"  \t模型預測\t\t差距量")
for index, number in enumerate(testDataNumbers):
  print (f"{index}\t{testDataPartition[index]}\t\t\t{int(normalizedPred[index])}\t=\t{abs(D[index])}")

In [ ]:
# for num in D:
#   print (abs(num))

## RMSE and error rate

In [ ]:
def calculate_mse(predictions, targets):
    """
    計算均方誤差 (MSE)

    參數:
    predictions: 預測值的列表或陣列
    targets: 實際值的列表或陣列，與預測值對應

    返回值:
    mse: 均方誤差
    """
    # 確保預測值和實際值的長度相等
    if len(predictions) != len(targets):
        raise ValueError("預測值和實際值的長度不一致")

    # 計算平方誤差
    squared_errors = [(p - t) ** 2 for p, t in zip(predictions, targets)]

    # 計算均方誤差
    mse = sum(squared_errors) / len(predictions)

    return mse

# 使用 LN
mse_result = calculate_mse(normalizedPred, testDataPartition)

# 使用 MLP(old)
#mse_result = calculate_mse(predictions, testDataPartition)

print("均方誤差 (MSE):", mse_result)


均方誤差 (MSE): [38138.887]


In [ ]:
mean = testData['Partition'].mean()

#LN MSE
MSE = int(calculate_mse(normalizedPred, testDataPartition))

#MLP MSE
# MSE = int(calculate_mse(predictions, testDataPartition))

RMSE= math.sqrt(MSE)
errorRate = RMSE/mean

print (f"MSE : {MSE}\nRMSE : {RMSE}\nError Rate : {errorRate}  ->  {round(errorRate,7)*100}%")

MSE : 38138
RMSE : 195.28952864913163
Error Rate : 0.01911043435259141  ->  1.9110399999999998%


<ipython-input-33-5f40d6fac6f7>:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  MSE = int(calculate_mse(normalizedPred, testDataPartition))


## calculate compare

In [ ]:
# G4 列表
g4_data = [
    12306.97217, 10763.92655, 11047.61962, 13718.37828, 16503.7549,
    16038.78061, 6888.485915, 11894.65019, 10896.95969, 12899.55966,
    10441.55882, 14573.09294, 7306.332787, 5688.120591, 10025.57947,
    14106.70248, 5521.838135, 8147.605931, 17884.49069, 7734.457538,
    11485.84299, 10807.51713, 7253.944225, 6511.296805, 13637.86933,
    13739.47874, 7585.445277, 2442.257115, 12530.05337, 16758.18477,
    19000.42027, 12834.74692, 9555.617669, 12661.66832, 2677.647078,
    7236.135417, 13226.24286, 8820.750395, 17036.26899, 16104.9424,
    6490.43907, 4201.908539, 10358.53232, 8464.027376, 8015.997274,
    16671.17028, 9933.079275, 10264.54316, 7099.194388, 8136.222586
]

# Partition 列表
partition_data = [
    8617, 19867, 8004, 10361, 23485, 12582, 10215, 17107, 7593, 9229,
    7271, 10311, 6145, 4055, 7413, 28467, 4255, 5706, 12475, 7447,
    9190, 8081, 5079, 5706, 11304, 9534, 10458, 1749, 8683, 14019,
    15744, 9222, 6730, 10344, 4979, 5024, 13112, 7834, 13176, 22661,
    9125, 3256, 7219, 5899, 7745, 29406, 9390, 8752, 11062, 5862
]


# data = [16566, 31717, 14594, 14832, 15673, 16346, 13108, 18098, 31284, 42050, 15623, 15886, 28364, 14047, 14997, 22961, 16070, 13302, 13996, 15871, 15845, 32717, 31204, 19410, 30918, 23276, 15798, 31307, 20215, 17037, 26490, 14456, 17861, 17657, 15340, 18648, 16372, 34817, 13763, 34670, 19071, 29963, 15154, 30274, 28296, 14357, 14977, 18535, 27127, 20936]
# Y = data

# data_G4 = [19959.77789, 19043.70527, 20192.60057, 19619.70493, 22696.4686, 21574.48907, 19045.11374, 21699.99089, 22538.3116, 22509.83347, 22120.12068, 21240.16567, 20326.67296, 20251.18902, 21669.83672, 22832.38546, 21528.54568, 19154.0269, 20081.63799, 19080.38587, 22729.65287, 21282.0421, 20075.35702, 20943.54237, 22309.57154, 22520.30163, 20152.6507, 22540.06135, 21866.0713, 19879.94198, 19099.09062, 20105.27236, 22373.46703, 19197.65336, 21991.31311, 21653.58805, 19698.68447, 20080.01293, 19680.65124, 22846.63572, 22202.19255, 21457.14545, 21858.58381, 20675.0703, 20419.95711, 20788.33579, 21550.50989, 22194.6816, 19650.14487, 22721.41732]
# G4 = data_G4

# data = [
#     41873, 15970, 36998, 16630, 41419, 16170, 36489, 46016, 21873, 46833,
#     24297, 18243, 20365, 35177, 20254, 16261, 16516, 17868, 51232, 47532,
#     18281, 24484, 38850, 22417, 34193, 16924, 18225, 40536, 43083, 18047,
#     26526, 18284, 35358, 36243, 19891, 18273, 18872, 16884, 21970, 18317,
#     21718, 33465, 16123, 16532, 32749, 59999, 25988, 17537, 17005, 46055
# ]


# data_G4 = [
#     25229.18863, 23054.15175, 26722.64269, 23835.95233, 26247.90106,
#     23151.84334, 26268.20655, 24954.14488, 26579.59727, 26389.03171,
#     25960.91309, 25967.55943, 26374.33224, 25472.36608, 24333.54365,
#     23460.68793, 23617.82988, 25390.8572, 26762.32906, 25375.47929,
#     24087.69113, 24771.50964, 23327.19795, 24150.18672, 23944.81831,
#     24381.84798, 24872.552, 26383.5254, 25867.54114, 25424.35725,
#     26170.15978, 25639.12247, 25451.47618, 25883.54246, 25625.28761,
#     26353.25729, 24555.51608, 23786.99037, 23820.59235, 26035.44141,
#     24111.96741, 24000.66944, 23183.25341, 23892.36305, 23438.92127,
#     26681.4039, 25428.99776, 23134.38886, 23625.46399, 24170.43602
# ]

# # G4 列表
# data = [
#     30582.87895, 28254.67602, 30779.91057, 29544.02103, 29884.46132,
#     27175.85977, 28656.286, 27580.47678, 27974.90818, 29539.97837,
#     30693.76277, 27482.21143, 29473.27854, 28413.85728, 30270.75728,
#     28134.31437, 28765.34852, 30554.06677, 29562.15822, 28413.82636,
#     30027.40146, 28375.44399, 30727.86066, 27146.52851, 29791.39657,
#     29695.26469, 29461.58446, 27369.92667, 27004.86417, 30008.71389,
#     27220.88625, 27377.96029, 30459.0978, 29327.84368, 27615.05727,
#     27746.74193, 27491.59142, 30045.41267, 30762.09074, 27040.28824,
#     30196.18223, 29532.55381, 30624.84745, 27616.09708, 30105.66893,
#     29775.09103, 29021.97298, 29846.00783, 28854.21881, 30041.17262
# ]

# # Partition 列表
# data_G4 = [
#     21247, 52497, 42538, 56567, 20655, 39674, 20618, 19672, 19776, 24851,
#     30866, 19145, 20381, 39672, 42856, 23547, 26713, 43392, 29272, 28543,
#     43559, 19590, 21548, 38724, 24732, 46592, 21568, 19175, 20544, 27743,
#     37742, 20150, 23361, 50096, 39452, 19807, 41416, 51226, 29715, 22527,
#     41906, 25002, 44874, 19909, 41788, 20557, 26829, 52046, 20804, 26579
# ]

# G4 列表
# data_G4 = [
#     30582.87895, 28254.67602, 30779.91057, 29544.02103, 29884.46132,
#     27175.85977, 28656.286, 27580.47678, 27974.90818, 29539.97837,
#     30693.76277, 27482.21143, 29473.27854, 28413.85728, 30270.75728,
#     28134.31437, 28765.34852, 30554.06677, 29562.15822, 28413.82636,
#     30027.40146, 28375.44399, 30727.86066, 27146.52851, 29791.39657,
#     29695.26469, 29461.58446, 27369.92667, 27004.86417, 30008.71389,
#     27220.88625, 27377.96029, 30459.0978, 29327.84368, 27615.05727,
#     27746.74193, 27491.59142, 30045.41267, 30762.09074, 27040.28824,
#     30196.18223, 29532.55381, 30624.84745, 27616.09708, 30105.66893,
#     29775.09103, 29021.97298, 29846.00783, 28854.21881, 30041.17262
# ]

# # Partition 列表
# data = [
#     21247, 52497, 42538, 56567, 20655, 39674, 20618, 19672, 19776, 24851,
#     30866, 19145, 20381, 39672, 42856, 23547, 26713, 43392, 29272, 28543,
#     43559, 19590, 21548, 38724, 24732, 46592, 21568, 19175, 20544, 27743,
#     37742, 20150, 23361, 50096, 39452, 19807, 41416, 51226, 29715, 22527,
#     41906, 25002, 44874, 19909, 41788, 20557, 26829, 52046, 20804, 26579
# ]



print ("G4 calculate:\n")
mean = testData['Partition'].mean()
MSE = int(calculate_mse(data_G4, data))
RMSE= math.sqrt(MSE)
errorRate = RMSE/mean

print (f"MSE : {MSE}\nRMSE : {RMSE}\nError Rate : {errorRate}  ->  {round(errorRate,7)*100}%")

# generate GoldBach Partition